# Section 1: Data analysis - linear regression.

In this section, we will practice basics tasks of curve fitting, using the linear regression approach. 

First lets load three modules
1. numpy - for fast computations with vectors
2. matplotlib - we will use this library to plot figures and graphs
3. h5py - this is used to save and read data from the `h5` format file (https://en.wikipedia.org/wiki/Hierarchical_Data_Format)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
%matplotlib inline

Assume we have a data set {x,y}. For this exercise the data is stored in `h5py` format file, which can be read using the next command:

In [ ]:
h5f = h5py.File('data/lindata.h5','r')
print("Keys: %s" % list(h5f.keys()))
x = h5f['x']
y = h5f['y']

For later proposes let's convert dimention of data vectors $\vec{v}$ from $n$ - dim to $n\times 1$ - dim

In [ ]:
print('x old shape = ',x.shape)
print('y old shape = ',y.shape)
x = np.expand_dims(x,axis=1)
y = np.expand_dims(y,axis=1)
print('x new shape = ',x.shape)
print('y new shape = ',y.shape)

Let's visualize the data, to do it we will use the `matplotlib.pyplot` function which is imported as `plt`

In [ ]:
plt.scatter(x,y,marker='o', c='k')
plt.xlabel('x'); plt.ylabel('y');
plt.xlim([0,2.3]); plt.ylim([0,100])
plt.show()

Our goal is to "learn" the trend in the data and to be able to predict what will be the outcome of a new data point.

As one can notice the data are more or less lying on a straight line 
$$y = ax + b$$

This will be our model.
What we want to compute is the model parameters $a$ and $b$.

How to choose the model parameters? Let's draw few examples for $(a,b) = (0,200)$ and $(a,b) = (4,50)$
First we define the model funciton:

In [ ]:
def model(x):
    return a*x + b

In [ ]:
a = 0
b = 50
x_line = np.linspace(0,100,100)
plt.scatter(x,y,marker='o', c='k')
plt.plot(x_line, model(x_line),c='r')
plt.xlabel('x'); plt.ylabel('y');
plt.xlim([0,2.3]); plt.ylim([0,100])
plt.show()

In [ ]:
a = 40
b = 2
x_line = np.linspace(0,100,100)
plt.scatter(x,y,marker='o', c='k')
plt.plot(x_line, model(x_line),c='r')
plt.xlabel('x'); plt.ylabel('y');
plt.xlim([0,2.3]); plt.ylim([0,100])
plt.show()

In our notation, we will call $y(x)$ as $h_{a,b}(x)$ as the letter $h$ stands for **<i>hypothesis</i>**

Today we have many data analysis frameworks that will do the job. In this exercise, we will use Linear Regression with gradient descent

### Cost function

Let's define a **cost function** as follows:

$$ J(a,b) = \frac{1}{2m}\Sigma_i \left( \hat{y}_i - y_i \right)^2$$

where $ \hat{y}_i = h_{a,b}(x) = ax_i + b $ and $m$ stands for total number of data points ($x_1,x_2,...,x_m$ and $y_1,y_2,...,y_m$)

As one can see, the **cost function** defines some "distance" between the predicted model and the actual data points.

In fitting problems, the idea is to minimize the cost function for $a$ and $b$ values.

First, let us define the **cost function**:



In [ ]:
def cost(a, b, x, y):

    m = len(x)    
    J = 0.5 * np.sum(np.square( a*x + b - y ) )  / m
   
    return J



Lets look how the cost function looks as a function of the model parameters in range $a\in(-1.5,2.5)$ and $b\in(-5,35)$


In [ ]:
m = len(x)        # m is the size of the sample
n_points = 400    # number of points in (a,b) space to scan
a_scan=np.expand_dims(np.linspace(-10, 90, num = n_points),axis=1)
b_scan=np.expand_dims(np.linspace(-50, 50, num = n_points),axis=1)

X,Y = np.meshgrid(a_scan, b_scan)
J_scan = np.zeros((n_points,n_points))
for i in range(n_points):
    for j in range(n_points):
        J_scan[i,j]=cost(X[i,j],Y[i,j],x,y)

plt.figure()
levels = np.logspace(0.1,5,20)
CS = plt.contour(X, Y, J_scan, levels=levels)
plt.clabel(CS, inline=1, fontsize=10)
plt.xlabel('a'); plt.ylabel('b'); plt.title('Contour plot of the cost function')
plt.show()


By eye it looks like for $a = 5$ and $b = 0$ we have minimal value of the cost function.

Lets Draw two predictions as before, one for the $a_1 = 0$ and $b_1 = 200$ and one for $a_2 = 4$ and $b_2 = 50$, and compute the total cost:


In [ ]:
a1 = 0; b1 = 50;
a2 = 40; b2 = 2;
plt.scatter(x,y,marker='o', c='k')
plt.plot(x,a1*x+b1,'r',label='y(x) = 0x + 50')
plt.plot(x,a2*x+b2,'m',label='y(x) = 40x + 2')
plt.xlabel('x'); plt.ylabel('y'); plt.legend(fontsize=16)
plt.show()
print('for a=0 and b=50 we compute J(a,b) = ', cost(a1,b1,x,y))
print('for a=40 and b=2 we compute J(a,b) = ', cost(a2,b2,x,y))

For this simple example, minimizing the cost function can be done analytically by solving two equations:

$$ \frac{\partial J}{\partial a}  = 0 $$

$$ \frac{\partial J}{\partial b}  = 0 $$

In practice (as we will see later) it is not always possible to calculate an exact solution.

We will use here one of the powerful numerical methods to minimize the cost function

### Gradient descent

The gradient descent aglorithm is defined as follows:

1. Compute partial derivatives of the cost function with respect to the model parameters: $\frac{\partial J}{\partial a}$ and $\frac{\partial J}{\partial b}$
2. Initialize weight to some specific values $(a,b) = (a_0,b_0)$
3. Define a **learning rate** parameter $\alpha$
4. Repeat $n_\text{iteration}$ times (latter we will denote $n_\text{iteration}$ as a number of **epoch**s)

$$ \theta_0^\text{temp} = \theta_0 - \alpha\times \frac{\partial J}{\partial \theta_0} $$ 

$$ \theta_1^\text{temp} = \theta_1 - \alpha\times \frac{\partial J}{\partial \theta_1} $$ 

$$ \theta_0 = \theta_0^\text{temp} $$ 

$$ \theta_1 = \theta_1^\text{temp} $$ 


here we change the notation from $(a,b)$ to ($\theta_0, \theta_1$)


### Partial derivative of the cost function:

Using the cost function define by:

$$ J(\theta) = \frac{1}{2m}\Sigma_i \left( \theta_0 + \theta_1x_i - y_i \right)^2$$

The partial derivative is given by

\begin{eqnarray}
\frac{\partial J}{\partial \theta_0} & = & \frac{1}{m}\Sigma_i \left( \theta_0 + \theta_1x_i - y_i \right) \\
\frac{\partial J}{\partial \theta_1} & = & \frac{1}{m}\Sigma_i \left( \theta_0 + \theta_1x_i - y_i \right)x_i \\
\end{eqnarray}

Let's define the cost function again, this time it will return the $J(\theta)$ and $\frac{\partial J}{\partial \theta}$

We will use vectorial representation as follows:
$$ ax+b = \theta_0 + \theta_1x = \begin{pmatrix}\theta_0 &&  \theta_1 \end{pmatrix} \cdot \begin{pmatrix} 1  \\ x \end{pmatrix} $$

Therefore we will update the input variable $x$ to $ [1 ;  x] $ and the weigts will be a column vector $\vec{\theta} = \begin{pmatrix}\theta_0 \\  \theta_1 \end{pmatrix}$

Using this formalizm the prediction $\hat{y}$ will be equal to $\hat{y} = x\cdot\theta$

In [ ]:
def cost(weights, x, y):
    
    m = len(x)
    x = np.append(np.ones(x.shape),x, axis=1)
    J = 0.5 * np.sum(np.square(np.matmul(x,weights) - y)) / m

    grad = np.sum( (np.matmul(x,weights) - y) * x, axis=0, keepdims = True).T / m
   
    return J, grad


### Implementation of the gradient descent alrorithm

Let's define the Gradient Descent function, with the following parameters:
- data $(X,Y)$
- initial weights (from now and on we will call denote the model parameters as weights)
- learning rate
- number of epochs

```python
def GradientDescent(X, Y, initial_weights, learning_rate, n_epoch):
    # code to calculate weights
    return weights, cache

```
We will also use a `cache` list to track the learning process of the algorithm (store the cost and the corresponding weights)

In [ ]:
def GradientDescent(X, Y, initial_weights, learning_rate, n_epoch):
    weights=initial_weights
    cache=[]
    for i in range(n_epoch):
        J, grad = cost(weights, X, Y)
        cache.append((J,weights))
        weights = weights - learning_rate*grad
    J, _ = cost(weights, X, Y)
    cache.append((J,weights))
    return weights, cache    

Let's run the gradient descent algorithm for 200 iterations (epochs), learning rate $\alpha=10^{-1}$ and look on the cost function:

In [ ]:
initial_weights = np.zeros((2,1))
weights, cache = GradientDescent(x, y, initial_weights, learning_rate = 0.1, n_epoch = 50)
print('after '+str(len(cache)-1)+' epochs the weights are a = '+str(weights[1])+' b = '+str(weights[0]))

Let's draw again the contour plot as before, but we will add the updated weights for each of the 5 iterations:

In [ ]:
nepoch = len(cache)
plt.figure(figsize=(15,7))
plt.subplot(121)
levels = np.logspace(0.1,5,20)
CS = plt.contour(X, Y, J_scan, levels=levels)
plt.clabel(CS, inline=1, fontsize=10)
plt.title('Contour plot of the cost function + updated weights for '+str(len(cache)-1)+' epochs')
plt.xlabel(r"a",fontsize=20); plt.ylabel(r"b",fontsize=20)
for i in range(nepoch):
    plt.scatter(cache[i][1][1],cache[i][1][0],color='r')
plt.subplot(122)
xepoch=np.linspace(0,nepoch-1,nepoch)
yepoch=np.zeros((nepoch,1))
for i in range(nepoch): yepoch[i]=cache[i][0]
plt.title('Evolution of the cost function')
plt.ylabel(r"$J(a,b)$",fontsize=20); plt.xlabel('number of epochs',fontsize=20)
plt.plot(xepoch,yepoch)
plt.show()

Try increase $\alpha$ and to see what is happens. Why for $\alpha=1$ the the algorithm diverges?
Try to increase number of steps (epochs) to $200$